# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd

import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, row_number
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format, to_timestamp, from_utc_timestamp, from_unixtime
from pyspark.sql.types import TimestampType

In [2]:
    spark_csv = SparkSession \
        .builder \
        .getOrCreate()

### Step 1: Scope the Project and Gather Data

#### Scope 
The purpose of this project is to collect about 2 million tweets from Twitter APIs, explored using Spark, cleaned via pandas and copy the clean data to S3, orchestrated the data by building the conceptual star schema model in Redshift using Airflow DAGs and Plugin Operators and performed necessary data quality checks thereafter to give to end users or data scientists for sentiment analysis of who will win 2020 US elections. 

#### Describe and Gather Data 
I have taken the dataset from Kaggle named US Elections 2020 Tweets that consists from about 1.72 Million Tweets in 2 csv files that are collected from various twitter APIs by screenscrapping the keywords "trump" and "biden" in the tweet text and collected the information into 2 csv files in columns as below:

created_at: Date and time of tweet creation
tweet_id: Unique ID of the tweet
tweet: Full tweet text
likes: Number of likes
retweet_count: Number of retweets
source: Utility used to post tweet
user_id: User ID of tweet creator
user_name: Username of tweet creator
user_screen_name: Screen name of tweet creator
user_description: Description of self by tweet creator
user_join_date: Join date of tweet creator
user_followers_count: Followers count on tweet creator
user_location: Location given on tweet creator's profile
lat: Latitude parsed from user_location
long: Longitude parsed from user_location
city: City parsed from user_location
country: Country parsed from user_location
state: State parsed from user_location
state_code: State code parsed from user_location
collected_at: Date and time tweet data was mined from twitter*

In [20]:
#Copied the 3 csv files in this project workspace and exploring the data in each using spark session.
f2name = './hashtag_donaldtrump.csv'
df2=spark_csv.read.format("csv").option("header", "true").option("delimiter",",").load(f2name)

In [4]:
#gives the schema or the columns in the csv file and their data types
df2.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- tweet_id: string (nullable = true)
 |-- tweet: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- retweet_count: string (nullable = true)
 |-- source: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- user_screen_name: string (nullable = true)
 |-- user_description: string (nullable = true)
 |-- user_join_date: string (nullable = true)
 |-- user_followers_count: string (nullable = true)
 |-- user_location: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- long: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- state: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- collected_at: string (nullable = true)



In [5]:
#gives the top 5 rows of the csv file to give you the sense on how the data looks like
df2.head(5)

[Row(created_at='2020-10-15 00:00:01', tweet_id='1.316529221557252e+18', tweet='#Elecciones2020 | En #Florida: #JoeBiden dice que #DonaldTrump solo se preocupa por él mismo. El demócrata fue anfitrión de encuentros de electores en #PembrokePines y #Miramar. Clic AQUÍ ⬇️⬇️⬇️', likes=None, retweet_count=None, source=None, user_id=None, user_name=None, user_screen_name=None, user_description=None, user_join_date=None, user_followers_count=None, user_location=None, lat=None, long=None, city=None, country=None, continent=None, state=None, state_code=None, collected_at=None),
 Row(created_at='⠀', tweet_id=None, tweet=None, likes=None, retweet_count=None, source=None, user_id=None, user_name=None, user_screen_name=None, user_description=None, user_join_date=None, user_followers_count=None, user_location=None, lat=None, long=None, city=None, country=None, continent=None, state=None, state_code=None, collected_at=None),
 Row(created_at='🌐https://t.co/qhIWpIUXsT', tweet_id=None, tweet=None, like

In [21]:
f4_name='./hashtag_joebiden.csv'
df4=spark_csv.read.format("csv").option("header", "true").option("delimiter",",").load(f4_name)

In [8]:
df4.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- tweet_id: string (nullable = true)
 |-- tweet: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- retweet_count: string (nullable = true)
 |-- source: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- user_screen_name: string (nullable = true)
 |-- user_description: string (nullable = true)
 |-- user_join_date: string (nullable = true)
 |-- user_followers_count: string (nullable = true)
 |-- user_location: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- long: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- state: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- collected_at: string (nullable = true)



In [9]:
df4.head(5)

[Row(created_at='2020-10-15 00:00:01', tweet_id='1.316529221557252e+18', tweet='#Elecciones2020 | En #Florida: #JoeBiden dice que #DonaldTrump solo se preocupa por él mismo. El demócrata fue anfitrión de encuentros de electores en #PembrokePines y #Miramar. Clic AQUÍ ⬇️⬇️⬇️', likes=None, retweet_count=None, source=None, user_id=None, user_name=None, user_screen_name=None, user_description=None, user_join_date=None, user_followers_count=None, user_location=None, lat=None, long=None, city=None, country=None, continent=None, state=None, state_code=None, collected_at=None),
 Row(created_at='⠀', tweet_id=None, tweet=None, likes=None, retweet_count=None, source=None, user_id=None, user_name=None, user_screen_name=None, user_description=None, user_join_date=None, user_followers_count=None, user_location=None, lat=None, long=None, city=None, country=None, continent=None, state=None, state_code=None, collected_at=None),
 Row(created_at='🌐https://t.co/qhIWpIUXsT', tweet_id=None, tweet=None, like

In [10]:
# creating a temporary view and then exploring the data using Spark SQL
df2.createOrReplaceTempView("trump_table")
spark_csv.sql('''
            select count(*) from trump_table
            ''').show()

# or
df2.count()

+--------+
|count(1)|
+--------+
| 2310698|
+--------+



2310698

In [11]:
df4.createOrReplaceTempView("biden_table")
spark_csv.sql('''
            select count(*) from biden_table
            ''').show()

# or
df4.count()

+--------+
|count(1)|
+--------+
| 1812041|
+--------+



1812041

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [3]:
trump_df=pd.read_csv(f2name,lineterminator='\n')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,2,3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# found out there are many columns which consists of many null values
trump_df.isnull().any()

In [ ]:
trump_df.head(5)

In [ ]:
biden_df=pd.read_csv(f4_name,lineterminator='\n')

In [ ]:
# found out there are many columns which consists of many null values
biden_df.isnull().any()

In [ ]:
biden_df.head(5)

In [ ]:
# used various methods as below:
#1
def func(x):
    try:
        return datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
    except:
        return pd.NaN

trump_df['created_at']=trump_df['created_at'].apply(func)
#2
trump_df['created_at']=pd.to_datetime(trump_df.created_at, errors='coerce')
#none of the above methods work

In [ ]:
# finally the below method worked for cleaning the csv files
#removing the error lines from 2 .csv files
trump_df = pd.read_csv("./hashtag_donaldtrump.csv",lineterminator='\n', error_bad_lines=False)
biden_df = pd.read_csv("./hashtag_joebiden.csv", error_bad_lines=False)
trump_df = trump_df[~trump_df.collected_at.isna()]
biden_df = biden_df[~biden_df.collected_at.isna()]

In [4]:
#from the above method, it removed the error lines or the bad data but still most of the columns of the 2 files have 
#NaN or Null values in multiple rows.
#handling each dataframe seperately, using bfill for backfill and ffill for forward fill assuming that the NaN or Null 
#values willbe filled with either the data of the same column of the row above or of the row below. 
trump_df=trump_df.bfill().ffill()

In [5]:
#since it is the csv file and by default csv file is comma seperated. commas in between the string of data present 
#in various columns were causing extra columns which is not required. For example, the tweet column in the csv file consists
#of the text that people wrote. This text can consists of multiple commas and since it is the csv file, commas in the single
#text can cause few extra columns in the file which is un-neccesary as well as incorrect.so replacing the commas with spaces
# in between the strings of text columns using the below
trump_df.replace(',',' ',regex=True,inplace=True)

In [6]:
trump_df.head(5)

created_at      tweet_id  \
0  2020-10-15 00:00:01  1.316529e+18   
1  2020-10-15 00:00:01  1.316529e+18   
2  2020-10-15 00:00:02  1.316529e+18   
3  2020-10-15 00:00:02  1.316529e+18   
4  2020-10-15 00:00:08  1.316529e+18   

                                               tweet  likes  retweet_count  \
0  #Elecciones2020 | En #Florida: #JoeBiden dice ...    0.0            0.0   
1  Usa 2020  Trump contro Facebook e Twitter: cop...   26.0            9.0   
2  #Trump: As a student I used to hear for years ...    2.0            1.0   
3  2 hours since last tweet from #Trump! Maybe he...    0.0            0.0   
4  You get a tie! And you get a tie! #Trump ‘s ra...    4.0            3.0   

               source       user_id              user_name user_screen_name  \
0           TweetDeck  3.606665e+08     El Sol Latino News  elsollatinonews   
1    Social Mediaset   3.316176e+08                Tgcom24  MediasetTgcom24   
2     Twitter Web App  8.436472e+06                 snarke           snarke   
3       Trumpytweeter  8.283556e+17          Trumpytweeter    trumpytweeter   
4  Twitter for iPhone  4.741380e+07  Rana Abtar - رنا أبتر        Ranaabtar   

                                    user_description  \
0  🌐 Noticias de interés para latinos de la costa...   
1  Profilo ufficiale di Tgcom24: tutte le notizie...   
2  Will mock for food! Freelance writer  blogger ...   
3  If he doesn't tweet for some time  should we b...   
4  Washington Correspondent  Lebanese-American  c...   

               ...               user_followers_count  \
0              ...                             1860.0   
1              ...                          1067661.0   
2              ...                             1185.0   
3              ...                               32.0   
4              ...                             5393.0   

                  user_location        lat        long        city  \
0  Philadelphia  PA / Miami  FL  25.774270  -80.193660    Portland   
1                      Portland  45.520247 -122.674195    Portland   
2                      Portland  45.520247 -122.674195    Portland   
3                 Washington DC  38.894992  -77.036558  Washington   
4                 Washington DC  38.894992  -77.036558  Washington   

                    country      continent                 state state_code  \
0  United States of America  North America               Florida         FL   
1  United States of America  North America                Oregon         OR   
2  United States of America  North America                Oregon         OR   
3  United States of America  North America  District of Columbia         DC   
4  United States of America  North America  District of Columbia         DC   

                    collected_at  
0            2020-10-21 00:00:00  
1  2020-10-21 00:00:00.373216530  
2  2020-10-21 00:00:00.746433060  
3  2020-10-21 00:00:01.119649591  
4  2020-10-21 00:00:01.492866121  

[5 rows x 21 columns]

In [7]:
#creating a csv file from the clean data
trump_df.to_csv('./hashtag_donaldtrump_updated_no_commas.csv')

In [8]:
#same steps performed for another dataset
biden_df=biden_df.bfill().ffill()

In [9]:
biden_df.replace(',','',regex=True,inplace=True)

In [11]:
biden_df.head(2)

created_at               tweet_id  \
0  2020-10-15 00:00:01  1.316529221557252e+18   
1  2020-10-15 00:00:18   1.31652929585929e+18   

                                               tweet likes  retweet_count  \
0  #Elecciones2020 | En #Florida: #JoeBiden dice ...   0.0            0.0   
1  #HunterBiden #HunterBidenEmails #JoeBiden #Joe...   0.0            0.0   

             source      user_id           user_name user_screen_name  \
0         TweetDeck  360666534.0  El Sol Latino News  elsollatinonews   
1  Twitter for iPad  809904438.0         Cheri A. 🇺🇸     Biloximeemaw   

                                    user_description  \
0  🌐 Noticias de interés para latinos de la costa...   
1  Locked and loaded Meemaw. Love God my family a...   

               ...               user_followers_count  \
0              ...                             1860.0   
1              ...                             6628.0   

                user_location      lat     long     city  \
0  Philadelphia PA / Miami FL  25.7743 -80.1937  Chicago   
1       Golden Valley Arizona   46.304 -109.171  Chicago   

                    country      continent    state state_code  \
0  United States of America  North America  Florida         FL   
1  United States of America  North America  Montana         MT   

                    collected_at  
0            2020-10-21 00:00:00  
1  2020-10-21 00:00:00.517827283  

[2 rows x 21 columns]

In [12]:
biden_df.to_csv('./hashtag_joebiden_updated_no_commas.csv')

In [2]:
# below steps are for writing the above 2 data frames in S3 in the form for csv files for fetching it to redshift. 
import pandas as pd

import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, row_number
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format, to_timestamp, from_utc_timestamp, from_unixtime
from pyspark.sql.types import TimestampType

In [5]:
# creating a dl.cfg file to contain the AWS Access KEY ID and AWS SECRET Access KEY to establish the connection between
#AWS S3 bucket and this jupyter notebook
config = configparser.ConfigParser()
config.read_file(open('dl.cfg'))

In [6]:
# creating the spark session using the values from the the above dl.cfg file
AWS_ACCESS_KEY_ID     = config.get('AWS', 'AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = config.get('AWS', 'AWS_SECRET_ACCESS_KEY')
spark = SparkSession \
        .builder \
        .config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11,org.apache.hadoop:hadoop-aws:2.7.2") \
        .config("fs.s3a.access.key", AWS_ACCESS_KEY_ID) \
        .config("fs.s3a.secret.key", AWS_SECRET_ACCESS_KEY) \
        .getOrCreate()

In [ ]:
# Aliter of above box
spark = SparkSession \
    .builder \
    .config("spark.jars.packages", "saurfang:spark-sas7bdat:2.1.0-s_2.11,org.apache.hadoop:hadoop-aws:2.7.0") \
    .enableHiveSupport() \
    .getOrCreate()

In [7]:
from pyspark.sql.types import *

In [8]:
# defining the structure of the csv file to be stored in S3 bucket
myschema=StructType([ StructField("created_at",StringType(),True)\
                     ,StructField("tweet_id",DoubleType(),True)\
                     ,StructField("tweet",StringType(),True)\
                     ,StructField("likes",DoubleType(),True)\
                     ,StructField("retweet_count",DoubleType(),True)\
                     ,StructField("source",StringType(),True)\
                     ,StructField("user_id",DoubleType(),True)\
                     ,StructField("user_name",StringType(),True)\
                     ,StructField("user_screen_name",StringType(),True)\
                     ,StructField("user_description",StringType(),True)\
                     ,StructField("user_join_date",StringType(),True)\
                     ,StructField("user_followers_count",DoubleType(),True)\
                     ,StructField("user_location",StringType(),True)\
                     ,StructField("lat",DoubleType(),True)\
                     ,StructField("long",DoubleType(),True)\
                     ,StructField("city",StringType(),True)\
                     ,StructField("country",StringType(),True)\
                     ,StructField("continent",StringType(),True)\
                     ,StructField("state",StringType(),True)\
                     ,StructField("state_code",StringType(),True)\
                     ,StructField("collected_at",StringType(),True)\
                    ])

In [9]:
trump_spark_df=spark.createDataFrame(trump_df,schema=myschema)
biden_spark_df=spark.createDataFrame(biden_df,schema=myschema)

In [ ]:
#writing the dataframe into S3 bucket path in the form of csv file
trump_spark_df.write.mode('overwrite')\
        .csv("s3a://udacity-capstone-project-us-elections-2020/hashtag_trump/hashtag_donaldtrump_updated_no_commas.csv", header=True)

In [ ]:
#same as above
biden_spark_df.write.mode('overwrite')\
        .csv("s3a://udacity-capstone-project-us-elections-2020/hashtag_biden/hashtag_joebiden_updated_no_commas.csv",header=True)

In [2]:
# aliter method of writing the .csv file in S3 bucket(fastest without creating a spark session and also 
#faster than uploading the .csv file directly in S3 web UI)
import boto3
session = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
)
s3 = session.resource('s3')
s3.meta.client.upload_file(Filename='./hashtag_donaldtrump_updated_no_commas.csv'
                           , Bucket='udacity-capstone-project-us-elections-2020'
                           , Key='boto3_hashtag_trump/hashtag_donaldtrump.csv')
s3.meta.client.upload_file(Filename='./hashtag_joebiden_updated_no_commas.csv'
                           , Bucket='udacity-capstone-project-us-elections-2020'
                           , Key='boto3_hashtag_biden/hashtag_joebiden.csv')

In [ ]:
#performing some more data exploration from the above 2 spark data frames
type(trump_spark_df)

In [ ]:
trump_spark_df.createOrReplaceTempView('trump_table')

In [ ]:
spark.sql('''
            SELECT created_at,tweet_id,tweet,user_id,retweet_count 
            FROM trump_table
            ''').show()

In [44]:
biden_spark_df.createOrReplaceTempView('biden_table')

In [ ]:
spark.sql('''
            SELECT created_at,tweet_id,tweet,user_id,retweet_count 
            FROM biden_table
            ''').show()

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.